https://github.com/BAMresearch/NFDI4IngScientificWorkflowRequirements

In [1]:
import os

In [2]:
import subprocess

In [3]:
def create_env(env_key):
    subprocess.check_output(
        ["mamba", "env", "create", "-n", env_key, "-f", env_dict[env_key], "-y"],
        universal_newlines=True,
    ).split("\n")

In [4]:
env_dict = {
    "preprocessing": "source/envs/preprocessing.yaml",
    "processing": "source/envs/processing.yaml",
    "postprocessing": "source/envs/postprocessing.yaml"
}

In [5]:
paper_source = "source/paper.tex"
macros_source = "source/macros.tex.template"

# Preprocessing 

In [6]:
env_key = "preprocessing"

In [7]:
create_env(env_key=env_key)

In [8]:
os.makedirs("preprocessing", exist_ok=True)

## generate mesh

In [9]:
domain_size = 2.0

In [10]:
gmsh_input = "source/unit_square.geo"
gmsh_output = "preprocessing/square.msh"

In [11]:
subprocess.check_output(
    ["conda", "run", "-n", env_key, "gmsh", "-2", "-setnumber", "domain_size", str(domain_size), gmsh_input, "-o", gmsh_output],
    universal_newlines=True,
).split("\n")

["Info    : Running 'gmsh -2 -setnumber domain_size 2.0 source/unit_square.geo -o preprocessing/square.msh' [Gmsh 4.6.0, 1 node, max. 1 thread]",
 'Info    : Started on Tue Jan 30 19:03:24 2024',
 "Info    : Reading 'source/unit_square.geo'...",
 'Used domain size: 2.000000',
 'Used mesh size: 0.500000',
 "Info    : Done reading 'source/unit_square.geo'",
 'Info    : Meshing 1D...',
 'Info    : [  0%] Meshing curve 1 (Line)',
 'Info    : [ 30%] Meshing curve 2 (Line)',
 'Info    : [ 50%] Meshing curve 3 (Line)',
 'Info    : [ 80%] Meshing curve 4 (Line)',
 'Info    : Done meshing 1D (Wall 0.00174594s, CPU 0.008912s)',
 'Info    : Meshing 2D...',
 'Info    : Meshing surface 1 (Plane, Frontal-Delaunay)',
 'Info    : Done meshing 2D (Wall 0.108933s, CPU 0.060695s)',
 'Info    : 98 nodes 198 elements',
 "Info    : Writing 'preprocessing/square.msh'...",
 "Info    : Done writing 'preprocessing/square.msh'",
 'Info    : Stopped on Tue Jan 30 19:03:24 2024 (From start: Wall 0.156303s, CPU 0.2

## convert to xdmf

In [12]:
meshio_output = "preprocessing/square.xdmf"

In [13]:
subprocess.check_output(
    ["conda", "run", "-n", env_key, "meshio", "convert", gmsh_output, meshio_output],
    universal_newlines=True,
).split("\n")

['']

# Processing

In [14]:
env_key = "processing"

In [15]:
create_env(env_key=env_key)

In [16]:
os.makedirs("processing", exist_ok=True)

## poisson

In [17]:
poisson_output_pvd =  "processing/poisson.pvd"
poisson_output_numdofs = "processing/numdofs.txt"

In [18]:
subprocess.check_output(
    ["conda", "run", "-n", env_key, "python", "source/poisson.py", "--mesh", meshio_output, "--degree", "2", "--outputfile", poisson_output_pvd, "--num-dofs", poisson_output_numdofs],
    universal_newlines=True,
).split("\n")

['Calling FFC just-in-time (JIT) compiler, this may take some time.',
 'Calling FFC just-in-time (JIT) compiler, this may take some time.',
 'Calling FFC just-in-time (JIT) compiler, this may take some time.',
 'Calling FFC just-in-time (JIT) compiler, this may take some time.',
 'Calling FFC just-in-time (JIT) compiler, this may take some time.',
 'Calling FFC just-in-time (JIT) compiler, this may take some time.',
 'Calling FFC just-in-time (JIT) compiler, this may take some time.',
 'Solving linear variational problem.',
 'Number of dofs used: 357',
 '',
 '']

# Postprocessing

In [19]:
env_key = "postprocessing"

In [20]:
create_env(env_key=env_key)

## plot over line

In [21]:
os.makedirs("postprocessing", exist_ok=True)

In [22]:
pvbatch_output = "postprocessing/plotoverline.csv"

In [23]:
subprocess.check_output(
    ["conda", "run", "-n", env_key, "pvbatch", "source/postprocessing.py", poisson_output_pvd, pvbatch_output],
    universal_newlines=True,
).split("\n")

['']

## substitute macros

In [24]:
with open(poisson_output_numdofs, "r") as instream:
    ndofs = int(instream.read())

In [25]:
macros_output = "macros.tex"

In [26]:
subprocess.check_output(
    ["conda", "run", "-n", env_key, "python", "source/prepare_paper_macros.py", "--macro-template-file", macros_source, "--plot-data-path", pvbatch_output, "--domain-size", str(domain_size), "--num-dofs", str(ndofs), "--output-macro-file", macros_output],
    universal_newlines=True,
).split("\n")

['']

## compile paper

In [27]:
paper_output = "paper.tex"

In [28]:
subprocess.check_output(
    ["cp", paper_source, paper_output],
    universal_newlines=True,
).split("\n")

['']

In [29]:
subprocess.check_output(
    ["conda", "run", "-n", env_key, "tectonic", paper_output],
    universal_newlines=True,
).split("\n")

['note: connecting to https://relay.fullyjustified.net/default_bundle.tar',
 'note: resolved to https://data1.fullyjustified.net/tlextras-2020.0r0.tar',
 'note: downloading index https://data1.fullyjustified.net/tlextras-2020.0r0.tar.index.gz',
 'note: downloading SHA256SUM',
 'note: generating format "latex"',
 'note: downloading tectonic-format-latex.tex',
 'note: downloading xelatex.ini',
 'note: downloading latex.ltx',
 'note: downloading texsys.cfg',
 'note: downloading fonttext.cfg',
 'note: downloading fonttext.ltx',
 'note: downloading omlenc.def',
 'note: downloading omsenc.def',
 'note: downloading ot1enc.def',
 'note: downloading t1enc.def',
 'note: downloading ts1enc.def',
 'note: downloading tuenc.def',
 'note: downloading tulmr.fd',
 'note: downloading tulmss.fd',
 'note: downloading tulmtt.fd',
 'note: downloading ts1lmr.fd',
 'note: downloading t1cmr.fd',
 'note: downloading ot1cmr.fd',
 'note: downloading ot1cmss.fd',
 'note: downloading ot1cmtt.fd',
 'note: downloadin

# Overview


In [30]:
! mamba env list

# conda environments:
#
base                     /srv/conda
notebook              *  /srv/conda/envs/notebook
postprocessing           /srv/conda/envs/postprocessing
preprocessing            /srv/conda/envs/preprocessing
processing               /srv/conda/envs/processing

